In [1]:
import random
import datetime
import numpy as np
import random

In [2]:
s=[65.0,90.0,2000.0,11] #可行初始解
tabu=[]#禁忌表
pareto=[] #pareto表
candidate=[]  #候选表

In [3]:
# 刹车盘质量，KG
def f1(s):
    res= 4.9*10**(-5)
    return res*(s[1]**2-s[0]**2)*(s[3]-1)
# 停止时间,s
def f2(s):
    res=9.82*10**(6)
    return res*(s[1]**2-s[0]**2)/(s[2]*s[3]*(s[1]**3-s[0]**3))
# 温升,N
def f3(s):
        return s[2]

In [4]:
tabu.append(s)
pareto.insert(0,[s,[f1(s),f2(s),f3(s)]])
tabu,pareto

([[65.0, 90.0, 2000.0, 11]],
 [[[65.0, 90.0, 2000.0, 11], [1.8987500000000002, 3.806677504064024, 2000.0]]])

In [5]:
# 边界控制
def edge(s):
    if s[0]<55 or s[0]>80:
        return True
    elif s[1]<75 or s[1]>110:
        return True
    elif s[2]<1000 or s[2]>3000:
        return True
    elif s[3]<2 or s[3]>20:
        return True
    else:
        return False

# 约束条件
def constraint(s):
    if (s[1]-s[0]-20)<0:
        return True
    elif (30-2.5*(s[3]+1))<0:
        return True
    elif (0.4-s[2]/(3.14*(s[1]**2-s[0]**2)))<0:
        return True
    elif (1-(2.22*(10**(-3))*s[2]*(s[1]**3-s[0]**3))/((s[1]**2-s[0]**2)**2))<0:
        return True
    elif (2.66/100*s[2]*s[3]*(s[1]**3-s[0]**3)/(s[1]**2-s[0]**2)-900)<0:
        return True
    else:
        return False
    
#     满足对于pareto表和candidate表中为pareto最优的解
def pareto_judge(pareto,s):
    pareto1 = np.array(pareto,dtype=object)
    i,flag,l_p=0,1,len(pareto1)
    
    while i<l_p:
        if ((s-pareto1[i][1])<=0).all():
            del pareto[i]
            pareto1 = np.delete(pareto1,[i],0)
            l_p-=1
        elif ((s-pareto[i][1])>=0).all():
            i+=1
            flag=0
        else:
            i+=1
    return flag
def candidate_judge(candidate,s):
    candidate1 = np.array(candidate,dtype=object)
    i,flag,l_c=0,1,len(candidate1)
    
    while i<l_c:
        if ((s-candidate1[i][1])<=0).all():
            del candidate[i]
            candidate1 = np.delete(candidate1,[i],0)
            l_c-=1
        elif ((s-candidate[i][1])>=0).all():
            i+=1
            flag=0
        else:
            i+=1
    return flag

# 相对于其他邻域解为pareto最优的解
def compare_peer(s):
    s=np.array(s,dtype=object)
    i,l=0,len(s)
    while i<l:
        j=i+1
        while j<l:
            if ((s[i][1]-s[j][1])<0).all():
                s=np.delete(s,[j],0)
                l-=1
            else:
                j+=1
        i+=1
    return s

In [6]:
def mots(length,s,tabu,candidate,pareto):

    for k in range(length):
            
        f, temp, can = [], [], []
        #         求得不在禁忌表中的邻域解

        for i in range(8):
            t = s.copy()
            if i < 4:
                t[int(i / 2)] += 0.1 * (-1) ** i
            elif i<6:
                t[int(i / 2)] += (-1) ** i
            else:
                t[int(i / 2)] += (-1) ** i
            temp.append(t)
        #          禁忌表及边界约束检查
        i, l = 0, len(temp)
        while i < l:
            if temp[i] in tabu:
                del temp[i]
                i -= 1
                l -= 1
            elif edge(temp[i]) or constraint(temp[i]):
                del temp[i]
                i -= 1
                l -= 1
            i += 1
        # 选出候选解
        for i in range(len(temp)):
            f.append([f1(temp[i]), f2(temp[i]), f3(temp[i])])
        f_np = np.array(f,dtype=object)
        for i in range(len(f_np)):
            if pareto_judge(pareto,f_np[i]) and candidate_judge(candidate,f_np[i]):
                can.append([i, f_np[i]])
        can = compare_peer(can)
        # 确定种子解，更新三个表
        if len(can) > 0:
#             pareto = pareto.tolist()
            seed=random.randint(0,len(can)-1)
            pareto.insert(0, [temp[can[seed][0]], can[seed][1]])
            s = temp[can[seed][0]]
            if len(tabu) < 20:
                tabu.insert(0, s)
            else:
                del tabu[19]
                tabu.insert(0, s)
#             del can[0]
            if len(can) > 1:
                for i in range(1,len(can)):
                    candidate.insert(0, [temp[can[i][0]], can[i][1]])
        #     无种子解，候选表有解，选出最老的候选解作为种子解并删除，更新三个表
        elif len(candidate) > 0:
            s = candidate.pop()[0]
            pareto.insert(0, [s, [f1(s), f2(s), f3(s)]])
            if len(tabu) < 20:
                tabu.insert(0, s)
            else:
                del tabu[19]
                tabu.insert(0, s)
        #      无种子解，候选表无解，终止算法   
        else:
            break
        print(pareto[0])

In [7]:
mots(20000,s,tabu,candidate,pareto)


[[65.0, 90.0, 1999.0, 11], array([1.8987500000000002, 3.808581794961505, 1999.0], dtype=object)]
[[64.9, 90.0, 1999.0, 11], array([1.9051150999999997, 3.8107352681699886, 1999.0], dtype=object)]
[[64.9, 90.0, 2000.0, 11], array([1.9051150999999997, 3.808829900535904, 2000.0], dtype=object)]
[[64.9, 90.0, 2001.0, 11], array([1.9051150999999997, 3.8069264373172453, 2001.0], dtype=object)]
[[65.0, 90.0, 2001.0, 11], array([1.8987500000000002, 3.804775116505771, 2001.0], dtype=object)]
[[65.0, 90.1, 2001.0, 11], array([1.9075748999999997, 3.8021024631487585, 2001.0], dtype=object)]
[[65.0, 90.1, 2000.0, 11], array([1.9075748999999997, 3.8040035143803324, 2000.0], dtype=object)]
[[65.0, 90.1, 1999.0, 11], array([1.9075748999999997, 3.805906467614139, 1999.0], dtype=object)]
[[65.0, 90.1, 1999.0, 10], array([1.7168174099999998, 4.186497114375554, 1999.0], dtype=object)]
[[65.1, 90.1, 1999.0, 10], array([1.7110800000000002, 4.184133860347083, 1999.0], dtype=object)]
[[65.19999999999999, 90.1,

[[66.39999999999992, 90.39999999999998, 1996.0, 11], array([1.8439680000000036, 3.773745476595337, 1996.0], dtype=object)]
[[66.39999999999992, 90.39999999999998, 1996.0, 10], array([1.6595712000000034, 4.151120024254871, 1996.0], dtype=object)]
[[66.39999999999992, 90.39999999999998, 1996.0, 9], array([1.4751744000000029, 4.612355582505413, 1996.0], dtype=object)]
[[66.39999999999992, 90.29999999999998, 1996.0, 9], array([1.468090960000003, 4.615551148914403, 1996.0], dtype=object)]
[[66.39999999999992, 90.19999999999999, 1996.0, 9], array([1.4610153600000038, 4.618750252010964, 1996.0], dtype=object)]
[[66.39999999999992, 90.19999999999999, 1996.0, 10], array([1.6436422800000043, 4.156875226809866, 1996.0], dtype=object)]
[[66.49999999999991, 90.19999999999999, 1996.0, 10], array([1.6377813900000042, 4.15452881027986, 1996.0], dtype=object)]
[[66.49999999999991, 90.19999999999999, 1996.0, 9], array([1.4558056800000037, 4.6161431225331775, 1996.0], dtype=object)]
[[66.49999999999991, 

[[68.19999999999982, 90.69999999999996, 1992.0, 7], array([1.0511235000000052, 5.870107513159762, 1992.0], dtype=object)]
[[68.19999999999982, 90.69999999999996, 1993.0, 7], array([1.0511235000000052, 5.867162150634342, 1993.0], dtype=object)]
[[68.29999999999981, 90.69999999999996, 1993.0, 7], array([1.0471104000000055, 5.8638659084481555, 1993.0], dtype=object)]
[[68.29999999999981, 90.59999999999997, 1993.0, 7], array([1.0417801800000057, 5.867851636214099, 1993.0], dtype=object)]
[[68.29999999999981, 90.59999999999997, 1993.0, 6], array([0.8681501500000048, 6.845826908916449, 1993.0], dtype=object)]
[[68.19999999999982, 90.59999999999997, 1993.0, 6], array([0.8714944000000046, 6.849679517896675, 1993.0], dtype=object)]
[[68.19999999999982, 90.59999999999997, 1994.0, 6], array([0.8714944000000046, 6.846244372702143, 1994.0], dtype=object)]
[[68.29999999999981, 90.59999999999997, 1994.0, 6], array([0.8681501500000048, 6.842393695822711, 1994.0], dtype=object)]
[[68.29999999999981, 90

[[67.69999999999985, 89.10000000000005, 1992.0, 10], array([1.4797843200000136, 4.166071129448073, 1992.0], dtype=object)]
[[67.69999999999985, 89.10000000000005, 1992.0, 9], array([1.315363840000012, 4.62896792160897, 1992.0], dtype=object)]
[[67.69999999999985, 89.20000000000005, 1992.0, 9], array([1.3223532000000118, 4.6257869813906805, 1992.0], dtype=object)]
[[67.59999999999985, 89.20000000000005, 1992.0, 9], array([1.3276569600000112, 4.6284315932296645, 1992.0], dtype=object)]
[[67.59999999999985, 89.20000000000005, 1993.0, 9], array([1.3276569600000112, 4.626109249229047, 1993.0], dtype=object)]
[[67.59999999999985, 89.20000000000005, 1993.0, 10], array([1.4936140800000126, 4.163498324306142, 1993.0], dtype=object)]
[[67.59999999999985, 89.10000000000005, 1993.0, 10], array([1.485751050000013, 4.166364110188363, 1993.0], dtype=object)]
[[67.69999999999985, 89.10000000000005, 1993.0, 10], array([1.4797843200000136, 4.163980777652062, 1993.0], dtype=object)]
[[67.69999999999985, 

[[68.69999999999979, 90.39999999999998, 1991.0, 10], array([1.5225392700000109, 4.107939737833818, 1991.0], dtype=object)]
[[68.79999999999978, 90.39999999999998, 1991.0, 10], array([1.5164755200000115, 4.105623517184966, 1991.0], dtype=object)]
[[68.89999999999978, 90.39999999999998, 1991.0, 10], array([1.510402950000012, 4.103308524198387, 1991.0], dtype=object)]
[[68.89999999999978, 90.49999999999997, 1991.0, 10], array([1.5183806400000115, 4.100534886906758, 1991.0], dtype=object)]
[[68.89999999999978, 90.49999999999997, 1991.0, 11], array([1.6870896000000126, 3.7277589880970536, 1991.0], dtype=object)]
[[68.79999999999978, 90.49999999999997, 1991.0, 11], array([1.693836900000012, 3.729859726258775, 1991.0], dtype=object)]
[[68.79999999999978, 90.49999999999997, 1992.0, 11], array([1.693836900000012, 3.7279873067174805, 1992.0], dtype=object)]
[[68.79999999999978, 90.49999999999997, 1992.0, 10], array([1.5244532100000108, 4.100786037389229, 1992.0], dtype=object)]
[[68.899999999999

...

[[69.89999999999972, 89.9, 1977.0, 7], array([0.9396240000000118, 5.88988782644198, 1977.0], dtype=object)]
[[69.89999999999972, 89.9, 1978.0, 7], array([0.9396240000000118, 5.886910127844183, 1978.0], dtype=object)]
[[69.89999999999972, 89.9, 1978.0, 6], array([0.7830200000000098, 6.868061815818214, 1978.0], dtype=object)]
[[69.89999999999972, 89.9, 1977.0, 6], array([0.7830200000000098, 6.871535797515643, 1977.0], dtype=object)]
[[69.89999999999972, 89.9, 1977.0, 5], array([0.6264160000000079, 8.245842957018771, 1977.0], dtype=object)]
[[69.89999999999972, 89.9, 1976.0, 5], array([0.6264160000000079, 8.250015954466654, 1976.0], dtype=object)]
[[69.89999999999972, 89.9, 1976.0, 4], array([0.4698120000000059, 10.31251994308332, 1976.0], dtype=object)]
[[69.89999999999972, 89.9, 1977.0, 4], array([0.4698120000000059, 10.307303696273465, 1977.0], dtype=object)]
[[69.89999999999972, 89.9, 1978.0, 4], array([0.4698120000000059, 10.30209272372732, 1978.0], dtype=object)]
[[69.89999999999972

[[69.99999999999972, 90.0, 1991.0, 7], array([0.9408000000000118, 5.8412374560429505, 1991.0], dtype=object)]
[[69.99999999999972, 90.0, 1992.0, 7], array([0.9408000000000118, 5.838305107922447, 1992.0], dtype=object)]
[[69.99999999999972, 90.0, 1992.0, 8], array([1.0976000000000137, 5.1085169694321415, 1992.0], dtype=object)]
[[69.99999999999972, 90.0, 1993.0, 8], array([1.0976000000000137, 5.105953739643164, 1993.0], dtype=object)]
[[69.99999999999972, 90.0, 1994.0, 8], array([1.0976000000000137, 5.103393080796803, 1994.0], dtype=object)]
[[69.99999999999972, 90.0, 1994.0, 9], array([1.2544000000000157, 4.536349405152714, 1994.0], dtype=object)]
[[69.99999999999972, 90.0, 1995.0, 9], array([1.2544000000000157, 4.53407554580176, 1995.0], dtype=object)]
[[69.99999999999972, 90.0, 1996.0, 9], array([1.2544000000000157, 4.53180396486699, 1996.0], dtype=object)]
[[69.99999999999972, 90.0, 1997.0, 9], array([1.2544000000000157, 4.529534658925644, 1997.0], dtype=object)]
[[69.99999999999972

[[69.99999999999972, 90.0, 1997.0, 3], array([0.31360000000000393, 13.58860397677693, 1997.0], dtype=object)]
[[69.99999999999972, 90.0, 1997.0, 4], array([0.4704000000000059, 10.1914529825827, 1997.0], dtype=object)]
[[69.99999999999972, 90.0, 1996.0, 4], array([0.4704000000000059, 10.196558920950727, 1996.0], dtype=object)]
[[69.99999999999972, 90.0, 1995.0, 4], array([0.4704000000000059, 10.20166997805396, 1995.0], dtype=object)]
[[69.99999999999972, 90.0, 1995.0, 5], array([0.6272000000000079, 8.161335982443168, 1995.0], dtype=object)]
[[69.99999999999972, 90.0, 1995.0, 6], array([0.7840000000000098, 6.801113318702639, 1995.0], dtype=object)]
[[69.99999999999972, 90.0, 1995.0, 7], array([0.9408000000000118, 5.82952570174512, 1995.0], dtype=object)]
[[69.99999999999972, 90.0, 1995.0, 8], array([1.0976000000000137, 5.10083498902698, 1995.0], dtype=object)]
[[69.99999999999972, 90.0, 1996.0, 8], array([1.0976000000000137, 5.098279460475363, 1996.0], dtype=object)]
[[69.99999999999972,

[[69.99999999999972, 90.0, 1994.0, 2], array([0.15680000000000197, 20.41357232318721, 1994.0], dtype=object)]
[[69.99999999999972, 90.0, 1995.0, 2], array([0.15680000000000197, 20.40333995610792, 1995.0], dtype=object)]
[[69.99999999999972, 90.0, 1996.0, 2], array([0.15680000000000197, 20.393117841901454, 1996.0], dtype=object)]
[[69.99999999999972, 90.0, 1997.0, 2], array([0.15680000000000197, 20.3829059651654, 1997.0], dtype=object)]
[[69.99999999999972, 90.0, 1997.0, 3], array([0.31360000000000393, 13.58860397677693, 1997.0], dtype=object)]
[[69.99999999999972, 90.0, 1998.0, 3], array([0.31360000000000393, 13.58180287368545, 1998.0], dtype=object)]
[[69.99999999999972, 90.0, 1999.0, 3], array([0.31360000000000393, 13.575008575099316, 1999.0], dtype=object)]
[[69.99999999999972, 90.0, 1999.0, 4], array([0.4704000000000059, 10.181256431324488, 1999.0], dtype=object)]
[[69.99999999999972, 90.0, 1998.0, 4], array([0.4704000000000059, 10.186352155264089, 1998.0], dtype=object)]
[[69.9999

[[70.49999999999969, 90.59999999999997, 2009.0, 5], array([0.6346695600000073, 8.04928001317794, 2009.0], dtype=object)]
[[70.59999999999968, 90.59999999999997, 2009.0, 5], array([0.6319040000000076, 8.044749849522285, 2009.0], dtype=object)]
[[70.59999999999968, 90.69999999999996, 2009.0, 5], array([0.6354574800000073, 8.039402356647566, 2009.0], dtype=object)]
[[70.59999999999968, 90.69999999999996, 2008.0, 5], array([0.6354574800000073, 8.043406043080159, 2008.0], dtype=object)]
[[70.59999999999968, 90.69999999999996, 2008.0, 6], array([0.7943218500000092, 6.702838369233466, 2008.0], dtype=object)]
[[70.59999999999968, 90.69999999999996, 2008.0, 7], array([0.953186220000011, 5.745290030771543, 2008.0], dtype=object)]
[[70.59999999999968, 90.69999999999996, 2008.0, 8], array([1.1120505900000128, 5.0271287769251, 2008.0], dtype=object)]
[[70.69999999999968, 90.69999999999996, 2008.0, 8], array([1.1072040000000132, 5.024302602803025, 2008.0], dtype=object)]
[[70.69999999999968, 90.6999

In [8]:
f_1,f_2,f_3=[],[],[]
for i in range(len(pareto)):
    f_1.append(pareto[i][1][0])
    f_2.append(pareto[i][1][1])
    f_3.append(pareto[i][1][2])
g_1,g_2,g_3=np.array(f_1),np.array(f_2),np.array(f_3)
print('x:',pareto[np.argmin(g_1)][0],'   min f1(x):',pareto[np.argmin(g_1)][1])
print('x:',pareto[np.argmin(g_2)][0],'  min f2(x):',pareto[np.argmin(g_2)][1])
print('x:',pareto[np.argmin(g_3)][0],'                min f3(x):',pareto[np.argmin(g_3)][1])

x: [69.69999999999973, 89.70000000000002, 1985.0, 2]    min f1(x): [0.156212000000002 20.58251063397876 1985.0]
x: [79.99999999999915, 101.39999999999935, 1965.0, 11]   min f2(x): [1.9021604000000025 3.323897588517763 1965.0]
x: [79.99999999999915, 102.3999999999993, 1943.0, 4]                 min f3(x): [0.600606719999999 9.1899846944577 1943.0]


In [9]:
len(pareto)

5866